# Lecture 13: GNIS

In [1]:
import numpy as np
import pandas as pd

---

# Scalar Functions and Query Plans

In [2]:
## we'll use the Lahman baseball database in our examples today.
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://localhost:5432/baseball
%config SqlMagic.displaylimit = 20

There's a new jupysql version available (0.10.14), you're running 0.10.0. To upgrade: pip install jupysql --upgrade
Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [5]:
%%sql
WITH year_num AS
  (SELECT year_id, (year_id % 100) as year
     FROM batting
  )
SELECT year_id, CONCAT('''', LPAD(year::text, 2, '0')) as year
  FROM year_num
 LIMIT 5;

Running query in 'postgresql://localhost:5432/baseball'

5 rows affected.

year_id,year
2004,'04
2007,'07
2009,'09
2010,'10
2012,'12


Let's analyze the below query (we've flattened it for convenience):

In [6]:
%%sql
EXPLAIN (VERBOSE true)
SELECT year_id,
       CONCAT('''', LPAD((year_id % 100)::text, 2, '0')) AS year
FROM batting;


Running query in 'postgresql://localhost:5432/baseball'

2 rows affected.

QUERY PLAN
Seq Scan on public.batting (cost=0.00..3922.29 rows=104324 width=36)
"Output: year_id, concat('''', lpad(((year_id % 100))::text, 2, '0'::text))"


What if scalar functions mention multiple tables?

The below query computes an arbitrary statistic for pitchers:
* 1 point for every strikeout they throw as pitcher
* –1 for every point they themselves struck out as batter

If the notebook-like output is hard to read, try out the query in `psql`. Note that notebooks don't preserve whitespace when displaying dataframes.

In [7]:
%%sql
EXPLAIN (VERBOSE true)
SELECT p.player_id, p.so - b.so
  FROM pitching p
  INNER JOIN batting b
  ON p.player_id=b.player_id;

Running query in 'postgresql://localhost:5432/baseball'

11 rows affected.

QUERY PLAN
Nested Loop (cost=0.43..12961.23 rows=336307 width=13)
"Output: p.player_id, (p.so - b.so)"
-> Seq Scan on public.pitching p (cost=0.00..1374.06 rows=45806 width=13)
"Output: p.player_id, p.year_id, p.stint, p.team_id, p.lg_id, p.w, p.l, p.g, p.gs, p.cg, p.sho, p.sv, p.ipouts, p.h, p.er, p.hr, p.bb, p.so, p.baopp, p.era, p.ibb, p.wp, p.hbp, p.bk, p.bfp, p.gf, p.r, p.sh, p.sf, p.gidp"
-> Memoize (cost=0.43..0.73 rows=7 width=13)
"Output: b.so, b.player_id"
Cache Key: p.player_id
Cache Mode: logical
-> Index Scan using batting_pkey on public.batting b (cost=0.42..0.72 rows=7 width=13)
"Output: b.so, b.player_id"


### Window Functions

In [8]:
%%sql
SELECT name_first, name_last, year_id, HR,
       rank() OVER (ORDER BY HR DESC),
       avg(HR)    OVER (PARTITION BY b.player_id ORDER BY year_id ROWS 3 PRECEDING) as avg_3yr,
       lag(HR, 7) OVER (PARTITION BY b.player_id ORDER BY year_id) as previous,
       lag(HR, 2) OVER (PARTITION BY b.player_id ORDER BY year_id) as lag2
FROM batting b, master m
WHERE m.player_id = b.player_id
   AND (name_last = 'Bonds' or name_last = 'Ruth')
ORDER BY HR DESC
LIMIT 10;

Running query in 'postgresql://localhost:5432/baseball'

RuntimeError: (psycopg2.errors.UndefinedTable) relation "master" does not exist
LINE 6: FROM batting b, master m
                        ^

[SQL: SELECT name_first, name_last, year_id, HR,
       rank() OVER (ORDER BY HR DESC),
       avg(HR)    OVER (PARTITION BY b.playerid ORDER BY year_id ROWS 3 PRECEDING) as avg_3yr,
       lag(HR, 7) OVER (PARTITION BY b.playerid ORDER BY year_id) as previous,
       lag(HR, 2) OVER (PARTITION BY b.playerid ORDER BY year_id) as lag2
FROM batting b, master m
WHERE m.player_id = b.player_id
   AND (name_last = 'Bonds' or name_last = 'Ruth')
ORDER BY HR DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


### Inverse Distribution Window Functions

In [11]:
%%sql
SELECT MIN(HR),
       percentile_cont(0.25) WITHIN GROUP (ORDER BY HR) AS p25,
       percentile_cont(0.50) WITHIN GROUP (ORDER BY HR) AS median,
       percentile_cont(0.75) WITHIN GROUP (ORDER BY HR) AS p75,
       percentile_cont(0.99) WITHIN GROUP (ORDER BY HR) AS p99,
       MAX(HR),
       AVG(HR) AS "average hit rate"
FROM batting;

Running query in 'postgresql://localhost:5432/baseball'

1 rows affected.

min,p25,median,p75,p99,max,average hit rate
0,0.0,0.0,2.0,31.0,73,2.8315823779763046


In [13]:
%%sql
SELECT HR, COUNT(*) FROM batting GROUP BY HR ORDER BY HR DESC;

Running query in 'postgresql://localhost:5432/baseball'

67 rows affected.

hr,count
73,1
70,1
66,1
65,1
64,1
63,1
61,1
60,1
59,2
58,3


### Hypothetical-Set Window Functions

In [14]:
hrs = 4 # hypothetically, four home runs

In [15]:
%%sql
SELECT {{hrs}} as hypothetical,
       rank({{hrs}}) WITHIN GROUP (ORDER BY HR DESC),
       dense_rank({{hrs}}) WITHIN GROUP (ORDER BY HR DESC),
       percent_rank({{hrs}}) WITHIN GROUP (ORDER BY HR DESC) * 100 AS pct_rank,
       cume_dist({{hrs}}) WITHIN GROUP (ORDER BY HR)
FROM batting
LIMIT 10;

Running query in 'postgresql://localhost:5432/baseball'

1 rows affected.

hypothetical,rank,dense_rank,pct_rank,cume_dist
4,18420,63,17.655573022506807,0.823445962137551


Without `jupysql` variable substituion

In [16]:
%%sql
SELECT 4 as hypothetical,
       rank(4) WITHIN GROUP (ORDER BY HR DESC),
       dense_rank(4) WITHIN GROUP (ORDER BY HR DESC),
       percent_rank(4) WITHIN GROUP (ORDER BY HR DESC) * 100 AS pct_rank,
       cume_dist(4) WITHIN GROUP (ORDER BY HR)
FROM batting
LIMIT 10;

Running query in 'postgresql://localhost:5432/baseball'

1 rows affected.

hypothetical,rank,dense_rank,pct_rank,cume_dist
4,18420,63,17.655573022506807,0.823445962137551


# GNIS

This notebook transforms the existing [Geographics Names Information Systems (GNIS)](https://www.usgs.gov/core-science-systems/ngp/board-on-geographic-names/download-gnis-data) national zip file.

We have provided a subset of the sql database for you in `data/national.sql`.

If you'd like to make your own version of this database, see the end of this notebook. Note: Because of its size, we don't recommend building the GNIS SQL database from scratch on DataHub.


In [18]:
!psql -h localhost -d gnis -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database() AND pid <> pg_backend_pid();'
!psql -h localhost -c 'DROP DATABASE IF EXISTS gnis'
!psql -h localhost -c 'CREATE DATABASE gnis' 
!psql -h localhost -d gnis -f data/gnis.sql

psql: error: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  database "gnis" does not exist
NOTICE:  database "gnis" does not exist, skipping
DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 3195
COPY 11533
CREATE INDEX


In [19]:
%reload_ext sql
%sql postgresql://localhost:5432/gnis
%config SqlMagic.displaylimit = 15

Connecting and switching to connection postgresql://localhost:5432/gnis

* View schema in `psql`
* View some rows below

In [20]:
%sql SELECT COUNT(*) FROM national;

Running query in 'postgresql://localhost:5432/gnis'

1 rows affected.

count
11533


In [21]:
%sql SELECT * FROM national WHERE county_name = 'Alameda';

Running query in 'postgresql://localhost:5432/gnis'

21 rows affected.

feature_id,feature_name,feature_class,state_alpha,state_numeric,county_name,county_numeric,primary_lat_dms,prim_long_dms,prim_lat_dec,prim_long_dec,source_lat_dms,source_long_dms,source_lat_dec,source_long_dec,elev_in_m,elev_in_ft,map_name,date_created,date_edited
218316,Apperson Creek,Stream,CA,6,Alameda,1.0,373349N,1215000W,37.5635453,-121.8332887,373232N,1214804W,37.5422222,-121.8011111,148.0,486.0,La Costa Valley,01/19/1981,None
225998,Irvington High School,School,CA,6,Alameda,1.0,373126N,1215801W,37.523814,-121.9670659,None,None,None,None,13.0,43.0,Niles,01/19/1981,03/31/2021
226951,Laurel Elementary School,School,CA,6,Alameda,1.0,374734N,1221147W,37.792899,-122.1964288,None,None,None,None,68.0,223.0,Oakland East,06/14/2000,03/14/2021
229367,Murray Elementary School,School,CA,6,Alameda,1.0,374318N,1215557W,37.721801,-121.9326269,None,None,None,None,112.0,367.0,Dublin,01/19/1981,03/14/2021
235581,Strawberry Creek,Stream,CA,6,Alameda,1.0,375221N,1221443W,37.8724258,-122.2452464,375251N,1221354W,37.8807588,-122.2316349,154.0,505.0,Oakland East,01/19/1981,08/31/2016
1654274,Hayward Golf Course,Locale,CA,6,Alameda,1.0,373726N,1220250W,37.6238222,-122.0471843,None,None,None,None,5.0,16.0,Newark,01/19/1981,None
1664964,KOFY-AM (San Mateo),Tower,CA,6,Alameda,1.0,374934N,1221842W,37.8260385,-122.3116366,None,None,None,None,2.0,7.0,Oakland West,07/01/1994,None
1670278,Lake Elizabeth,Lake,CA,6,Alameda,1.0,373255N,1215742W,37.5487056,-121.9617554,None,None,None,None,16.0,52.0,Niles,11/09/1995,03/07/2019
1692819,California School for the Deaf - Fremont,School,CA,6,Alameda,1.0,373334N,1215747W,37.5593966,-121.9631843,None,None,None,None,20.0,66.0,Niles,05/08/1996,09/16/2016
1692863,J A Freitas Library,Building,CA,6,Alameda,1.0,374335N,1220925W,37.7263185,-122.1569101,None,None,None,None,19.0,62.0,San Leandro,05/08/1996,None


In [22]:
%%sql
SELECT *
FROM national TABLESAMPLE BERNOULLI(10);

Running query in 'postgresql://localhost:5432/gnis'

1180 rows affected.

feature_id,feature_name,feature_class,state_alpha,state_numeric,county_name,county_numeric,primary_lat_dms,prim_long_dms,prim_lat_dec,prim_long_dec,source_lat_dms,source_long_dms,source_lat_dec,source_long_dec,elev_in_m,elev_in_ft,map_name,date_created,date_edited
3039,Clover Tank,Reservoir,AZ,4,Gila,7.0,335104N,1101528W,33.8509816,-110.2577249,None,None,None,None,1563.0,5128.0,Carrizo SE,02/08/1980,03/19/2019
3060,Coat Spring,Spring,AZ,4,Navajo,17.0,360736N,1101959W,36.12678,-110.3330424,None,None,None,None,1926.0,6319.0,Coat Spring,02/08/1980,04/18/2011
5427,Gunsight Canyon,Valley,AZ,4,Mohave,15.0,344716N,1133708W,34.7877893,-113.6188253,344718N,1134658W,34.7883432,-113.7827192,642.0,2106.0,Tule Wash,02/08/1980,None
7221,Little Water,Area,AZ,4,Navajo,17.0,352649N,1103415W,35.4469546,-110.5709674,None,None,None,None,1583.0,5194.0,Montezumas Chair,02/08/1980,None
8053,Milkweed Spring,Spring,AZ,4,Mohave,15.0,353706N,1134223W,35.6184467,-113.7063277,None,None,None,None,1441.0,4728.0,Milkweed Canyon SW,02/08/1980,04/18/2011
8884,Oak Spring,Spring,AZ,4,Mohave,15.0,364903N,1133510W,36.8174811,-113.5860693,None,None,None,None,1605.0,5266.0,Wolf Hole Mountain East,02/08/1980,None
9673,Post Office Farms,Bend,AZ,4,Navajo,17.0,335733N,1095503W,33.9592139,-109.917602,None,None,None,None,1759.0,5771.0,Alchesay Flat,02/08/1980,None
10197,Reese Ranch,Locale,AZ,4,Gila,7.0,330510N,1104244W,33.0861745,-110.7123268,None,None,None,None,633.0,2077.0,Christmas,02/08/1980,None
21598,Dead River Bridge,Bridge,AZ,4,Apache,1.0,350505N,1094224W,35.0847456,-109.7067815,None,None,None,None,1686.0,5531.0,Pinta,06/27/1984,None
24724,Bethany Villa Adult Mobile Home Park,Populated Place,AZ,4,Maricopa,13.0,333134N,1120951W,33.5261527,-112.1640412,None,None,None,None,352.0,1155.0,Glendale,06/27/1984,None


# Numerical Granularity

In [23]:
%sql SELECT elev_in_m FROM National LIMIT 2;

Running query in 'postgresql://localhost:5432/gnis'

2 rows affected.

elev_in_m
931.0
2707.0


In [27]:
%%sql
SELECT elev_in_m, 
    (elev_in_m / 100)::INTEGER AS quantized,
    ((elev_in_m / 100)::INTEGER) * 100 AS round_to_100,
    SUBSTRING(elev_in_m::TEXT, 1, 2),
    CONCAT(SUBSTRING(elev_in_m::TEXT, 1, 2), '00') AS substring2
FROM National
LIMIT 5;

Running query in 'postgresql://localhost:5432/gnis'

5 rows affected.

elev_in_m,quantized,round_to_100,substring,substring2
931.0,9,900,93,9300
2707.0,27,2700,27,2700
275.0,3,300,27,2700
1685.0,17,1700,16,1600
1354.0,14,1400,13,1300


In [28]:
%config SqlMagic.named_parameters=True

In [30]:
%%sql
/* does not run in jupysql b/c of <<. copy into psql  */
SELECT elev_in_m,
    (16::INTEGER::BIT(12)) AS bit12, 
    (16::INTEGER::BIT(12)) << 3
FROM national
LIMIT 5;

Running query in 'postgresql://localhost:5432/gnis'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(psycopg2.errors.SyntaxError) syntax error at or near "3"
LINE 1: 3
        ^

[SQL: 3
FROM national
LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
%%sql
EXPLAIN (verbose)
WITH shifts AS (
SELECT elev_in_m, 
       (elev_in_m::integer::bit(12)) AS bit12, 
       (elev_in_m::integer::bit(12) >> 8) AS rightshifted, 
       ((elev_in_m::integer::bit(12) >> 8) << 8)::integer AS round_to_256,
       ((elev_in_m::integer::bit(12) >> 8) << 8)::integer % 256 AS test
  FROM national
)
SELECT COUNT(DISTINCT elev_in_m) AS elevation_meters_count,
       COUNT(DISTINCT bit12) AS bit12_count,
       COUNT(DISTINCT rightshifted) AS rightshift_count,
       COUNT(DISTINCT round_to_256) AS rounded_count
  FROM shifts;

# Demo 1: Roll-up / Drill-down Practice

Let's start with county-level data on elevations:

In [ ]:
%%sql
SELECT state_numeric, county_numeric,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
FROM national TABLESAMPLE BERNOULLI(10)
GROUP BY state_numeric, county_numeric;

**Roll up** to state level.
* We save the view as `state_elevations` for later...

In [ ]:
%%sql
DROP VIEW IF EXISTS state_elevations;

CREATE VIEW state_elevations AS
SELECT state_numeric,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
FROM national 
GROUP BY state_numeric
;

In [ ]:
%sql SELECT * FROM state_elevations;

**Drill down** to include feature class.

In [ ]:
%%sql
SELECT state_numeric, feature_class,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
FROM national TABLESAMPLE Bernoulli(10)
GROUP BY state_numeric, feature_class
ORDER BY count(*) DESC;

# Demo 2: Connections to Statistics

## Roll up with marginal distributions

In [ ]:
%%sql
SELECT state_numeric,
       AVG(elev_in_m),
       STDDEV(elev_in_m), COUNT(*),
       SUM(COUNT(*)) OVER () AS total,
       COUNT(*)/SUM(COUNT(*)) OVER () AS marginal
FROM national TABLESAMPLE Bernoulli(.07)
GROUP BY state_numeric;

In [ ]:
%%sql
SELECT COUNT(DISTINCT county_numeric) FROM national;

## Drill down with normally-distributed elevations:

Start with the `state_elevations` view from earlier:

In [ ]:
%sql SELECT * FROM state_elevations;

The `fips_counties` relation has all counties, including those not in `national`:

In [ ]:
%sql SELECT * FROM fips_counties LIMIT 10;

If we wanted to **drill down** to the FIPS counties, we'd need to simulate an elevation for those counties that don't exist in `national`.

Here's the first step in that process, which creates a simulated value for *every* county in `fips_counties`.
* The value is simulated from a normal distribution using that state's elevation statistics (average, standard deviation).
* Just like a Python package, we would need to import `tablefunc` in order to use the `normal_rand` function.

In [ ]:
%sql CREATE EXTENSION IF NOT EXISTS tablefunc;

In [ ]:
%%sql
WITH state_cty AS
(SELECT s.state_numeric, f.fips as county_numeric, s.avg, s.stddev, s.count
  FROM state_elevations s, fips_counties f
  WHERE s.state_numeric = f.state_numeric
)
SELECT s.*,
       n.n AS elev_in_m,
       true as elev_in_m_sim -- user-facing flag
  FROM state_cty s,
       LATERAL normal_rand(CAST(s.count AS INTEGER), s.avg, s.stddev) AS n
LIMIT 10;

# Assembling an Explicit Hierarchy

In [ ]:
## we'll use the Lahman baseball database in our initial examples today.
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://localhost:5432/baseball

Two relations have the pieces of the hierarchy we want:

In [ ]:
%sql SELECT * FROM Appearances WHERE yearid > 1970 LIMIT 2;

In [ ]:
%sql SELECT * FROM Teams LIMIT 1;

Let's join these two to make our hierarchy! Which way should we make this?

In [ ]:
%%sql
SELECT a.playerid, a.teamid, t.divid, a.*
  FROM Appearances a
  NATURAL JOIN Teams t
WHERE a.yearid = 2015
LIMIT 100;

In [ ]:
%%sql
CREATE OR REPLACE VIEW bball_tree AS
SELECT DISTINCT a.playerid, a.teamid, t.divid, a.lgid, a.yearid
  FROM Appearances a NATURAL JOIN Teams t;

### Revisiting the Home Run Query

Recall our old home run query:

In [ ]:
%%sql
SELECT namefirst, namelast, yearid,
       MIN(hr), MAX(hr), AVG(hr), STDDEV(hr), SUM(hr)
FROM batting b, people p
WHERE b.playerid = p.playerid
GROUP BY namelast, namefirst, yearid
ORDER BY max DESC
LIMIT 10;

Set up for roll up/drill down on `bball_tree` hierarchy.
* Join each (raw) person with the associated `bball_tree` entry by `(playerid, yearid)` in a CTE
* Use this result for roll-up and drill-down.

<br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/>

In [ ]:
%%sql
WITH batting_tree AS
(
    SELECT b.*, t.divid
    FROM batting b, bball_tree t
    WHERE b.playerid = t.playerid
      AND b.yearid = t.yearid
)
SELECT namefirst, namelast,
       bt.teamid, bt.lgid, bt.divid, bt.yearid,
       MIN(hr), MAX(hr), AVG(hr), STDDEV(hr), SUM(hr)
FROM batting_tree bt, people p
WHERE bt.playerid = p.playerid
GROUP BY bt.playerid, bt.teamid, bt.lgid, bt.divid, bt.yearid, namelast, namefirst
ORDER BY max DESC
LIMIT 10;

# [Extra] Load in the database from scratch

We download the database, unzip it, load it into pandas, then export to a new database via `jupysql` cell magic.

**CAUTION**: This may crash your DataHub instance. The file is pretty big....

The direct zip download of this file is [here](https://geonames.usgs.gov/docs/stategaz/NationalFile.zip).

In [ ]:
# first download and unzip the data
!mkdir -p data
!wget https://geonames.usgs.gov/docs/stategaz/NationalFile.zip -P data/
!unzip -u data/NationalFile.zip -d data/

In [ ]:
import os
fname = os.path.join("data", "NationalFile_20210825.txt")
fname

In [ ]:
!du -h {fname} # big file

In [ ]:
!head -c 1024 {fname}

In [ ]:
# next, load it into pandas
import pandas as pd

national = pd.read_csv("data/NationalFile_20210825.txt", delimiter="|")
national.head(2)

In [ ]:
national = national.rename(columns=dict([(col, col.lower().strip()) for col in national.columns]))
national.head(2)

Next, get a table sample in pandas.

In [ ]:
import numpy as np

p = 0.005 # fraction, not percentage

np.random.seed(42)
national['keep_bool'] = np.random.random(len(national)) < p
national['keep_bool'].value_counts()

In [ ]:
national = national[national['keep_bool']].drop(columns=['keep_bool'])
national

Now, export to SQL

In [ ]:
!psql -h localhost -d gnis -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database() AND pid <> pg_backend_pid();'
!psql -h localhost -c 'DROP DATABASE IF EXISTS gnis'
!psql -h localhost -c 'CREATE DATABASE gnis' 

In [ ]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/gnis

In [ ]:
%sql --persist-replace national

In [ ]:
%sql ALTER TABLE national DROP COLUMN index;

Now, export to file with `pgdump`

In [ ]:
!pg_dump -h localhost --encoding utf8 gnis -f data/gnis.sql 

Finally, run the beginning of this notebook again

In [ ]:
!du -h data/gnis.sql

## FIPS

Federal Information Processing System (FIPS) Codes for States and Counties

Manually download the file from this link (https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt) and save it in `data/`.
* `wget` does not work here; likely the FCC website only accepts HTTPS connections to deter from server attacks.

In [ ]:
!wget https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt -P data/

In [ ]:
import pandas as pd
import re

In [ ]:
with open('data/fips.txt', 'r') as f:
    lines = f.readlines()

In [ ]:
COUNTY_STARTS = 69
OFFSET = 3 # the start of the data itself, after headers

In [ ]:
re.match('\s+(\d+)\s+(\w+)', lines[COUNTY_STARTS+3]).groups()

In [ ]:
splits = [re.match('\s+(\d+)\s+(.*)', line).groups()
          for line in 
          lines[COUNTY_STARTS+OFFSET:]]
splits[0]

**For later**: There is a significant discrepancy between the number of counties created and the number of lines remaining in our dataset. We encourage you to investigate this!

In [ ]:
len(lines), len(splits)

> FIPS codes are numbers which uniquely identify geographic areas.  The number of 
digits in FIPS codes vary depending on the level of geography.  State-level FIPS
codes have two digits, county-level FIPS codes have five digits of which the 
first two are the FIPS code of the state to which the county belongs.  When 
using the list below to look up county FIPS codes, it is advisable to first look
up the FIPS code for the state to which the county belongs.  This will help you
identify the right section of the list while scrolling down, which can be
important since there are over 3000 counties and county-equivalents (e.g.
independent cities, parishes, boroughs) in the United States.

In [ ]:
fips_counties = pd.DataFrame(data=splits, columns=['fips', 'county'])
fips_counties['state_numeric'] = fips_counties['fips'].str[:2].astype(int)
fips_counties['fips'] = fips_counties['fips'].astype(int)
fips_counties = fips_counties.set_index('fips')
fips_counties

In [ ]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/gnis

In [ ]:
%sql --persist-replace fips_counties

Now, export to file with `pgdump`. This exports both `national` and `fips_counties` relations to the same `gnis.sql` database dump.

In [ ]:
!pg_dump -h localhost --encoding utf8 gnis -f data/gnis.sql 

Finally, run the beginning of this notebook again

In [ ]:
!du -h data/gnis.sql